Import dataset

In [1]:
import numpy as np
import torch
from data.dataset import DatasetLoader
from data.data import TorchGraphData
from preprocessing.normalize import *
from preprocessing.batching_v2 import get_batch_graphs
import math
from typing import List

def flowrate_bc_sin(flowrate, n_edge : int, 
                    n_time : int)->torch.Tensor:
    _flowrate = []
    T = 4.8 # seconds
    for i in range(n_time):
        t = i * T / (n_time - 1)
        value = flowrate(t)
        _flowrate.append(torch.full(size=(n_edge, 1), fill_value=value))
    return torch.cat(_flowrate, dim=1)
        

def normalize(data, mean_ea, std_ea):
    x = min_max_scaler(data.x, min=0, max=500)
    edge_index = data.edge_index
    edge_attr = standard_scaler(data.edge_attr,
                mean=mean_ea,
                std=std_ea)
    pressure = data.pressure*1e-3
    velocity = data.velocity*1e-3
    f = lambda t : (np.pi * 1e0 / 4.8) * math.sin(2 * np.pi * t / 4.8)
    flowrate_bc = flowrate_bc_sin(flowrate=f, n_edge=data.edge_attr.size(0),\
                                n_time=data.flowrate.size(1))
    return TorchGraphData(x=x,edge_index=edge_index,edge_attr=edge_attr,
                        pressure=pressure, velocity=velocity, 
                        flowrate_bc=flowrate_bc)

def train_eval_split(dataset : DatasetLoader, train_id : List, eval_id : List,
                    batch_size : int, batch_n_times : int, recursive : bool):
    mean, std = mean_std_dataset(dataset, set_id=list(range(dataset.len())))
    # Train dataset
    train_dataset = []
    for i in train_id:
        data = normalize(dataset[i], mean, std)
        train_dataset += get_batch_graphs(data=data, batch_size=batch_size,
                        batch_n_times=batch_n_times, recursive=recursive)
    # Test dataset
    eval_dataset = []
    for i in eval_id:
        data = normalize(dataset[i], mean, std)
        eval_dataset += get_batch_graphs(data=data, batch_size=batch_size,
                        batch_n_times=batch_n_times, recursive=recursive)
        # eval_dataset += [data]

    return train_dataset, eval_dataset
def mean_std_dataset(dataset : DatasetLoader, set_id : List, field : str = 'edge_attr'):
    fields = []
    for i in set_id:
        fields.append(dataset[i]._store[field])
    fields = torch.cat(fields, dim=0)
    return fields.mean(axis=0), fields.std(axis=0)

In [2]:
_dataset = DatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_transformed'
)

train_dataset, eval_dataset = train_eval_split(
    dataset=_dataset,
    train_id=list(range(0, 20)),
    eval_id=list(range(20, 40)),
    batch_size=10000,
    batch_n_times=10,
    recursive=True
)

Train

In [3]:
import os
import torch
from networks.network_recurrent import RecurrentMeshGraphNet
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from networks.network_recurrent import objectview
CUDA_LAUNCH_BLOCKING=1
torch.cuda.empty_cache()

# Model params
args = objectview({
    'input_dim_node' : _dataset[0].x.size(1)+1,
    'input_dim_edge' : _dataset[0].edge_attr.size(1)+2,
    'output_dim_node' : 1,
    'output_dim_edge' : 1,
    'hidden_dim' : 128,
    'n_processors' : 10,
    'n_time' : _dataset[0].pressure.size(1),
    'device' : torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'lr' : 1e-7,
    'weight_decay' : 5e-2,
    'epoch' : 100
})

# Model initializing
model = RecurrentMeshGraphNet(
    input_dim_node = args.input_dim_node,
    input_dim_edge = args.input_dim_edge,
    output_dim_node = args.output_dim_node,
    output_dim_edge = args.output_dim_edge,
    hidden_dim = args.hidden_dim,
    n_processors = args.n_processors
)
model = model.to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
criterion = torch.nn.MSELoss()


In [4]:
# Train function v2
def train(model, data, args):
    n_edge = data.edge_attr.size(0)
    hidden = torch.zeros(n_edge, args.hidden_dim).to(args.device)
    _x = data.x #.to(args.device)
    edge_index = data.edge_index.to(args.device)
    _edge_attr = data.edge_attr #.to(args.device)

    _pressure = data.pressure[:,0].unsqueeze(1)
    _velocity = data.velocity[:,0].unsqueeze(1)

    loss = 0
    for i in range(1, data.number_of_timesteps):
        model.train()
        optimizer.zero_grad()

        # _pressure = data.pressure[:,i-1].unsqueeze(1)
        # _velocity = data.velocity[:,i-1].unsqueeze(1)
        _flowrate_bc = data.flowrate_bc[:,i].unsqueeze(1)
        x = torch.cat([_x, _pressure], dim=1).to(args.device)
        edge_attr = torch.cat([_edge_attr, _velocity, _flowrate_bc], dim=1).to(args.device)
        node_out, edge_out, hidden = model(x, edge_index, edge_attr, hidden)
        hidden = hidden.detach()
        _pressure = node_out.detach().cpu()
        _velocity = edge_out.detach().cpu()

        loss += criterion(node_out, data.pressure[:,i].unsqueeze(1).float().to(args.device))
        loss += criterion(edge_out, data.velocity[:,i].unsqueeze(1).float().to(args.device))

    loss.backward()
    optimizer.step()
    return loss.item()

# Eval function
def eval(model, data, args):
    n_edge = data.edge_attr.size(0)
    hidden = torch.zeros(n_edge, args.hidden_dim).to(args.device)
    _x = data.x #.to(args.device)
    edge_index = data.edge_index.to(args.device)
    _edge_attr = data.edge_attr #.to(args.device)

    _pressure = data.pressure[:,0].unsqueeze(1)
    _velocity = data.velocity[:,0].unsqueeze(1)

    loss = 0
    for i in range(1, data.number_of_timesteps):
        model.eval()
        with torch.no_grad():
            # _pressure = data.pressure[:,i-1].unsqueeze(1)
            # _velocity = data.velocity[:,i-1].unsqueeze(1)
            _flowrate_bc = data.flowrate_bc[:,i].unsqueeze(1)
            x = torch.cat([_x, _pressure], dim=1).to(args.device)
            edge_attr = torch.cat([_edge_attr, _velocity, _flowrate_bc], dim=1).to(args.device)
            node_out, edge_out, hidden = model(x, edge_index, edge_attr, hidden)
            hidden = hidden.detach()
            _pressure = node_out.detach().cpu()
            _velocity = edge_out.detach().cpu()

            loss += criterion(node_out, data.pressure[:,i].unsqueeze(1).float().to(args.device))
            loss += criterion(edge_out, data.velocity[:,i].unsqueeze(1).float().to(args.device))

    return loss.item()
# Training
total_train_loss = []
total_eval_loss = []
for epoch in range(args.epoch):
    train_loss = 0
    for data in train_dataset:
        train_loss += train(model=model, data=data, args=args)
    train_loss /= len(train_dataset)
    total_train_loss.append(train_loss)

    eval_loss = 0
    for data in eval_dataset:
        eval_loss += eval(model=model, data=data, args=args)
    eval_loss /= len(eval_dataset)
    total_eval_loss.append(eval_loss)
    
    print(f'Epoch {epoch}: train loss = {train_loss}; eval loss = {eval_loss}')
    if epoch % 20 == 0:
        torch.save(model.state_dict(), f'models/rmgnv1_epoch{epoch}.pth')
    
    

Epoch 0: train loss = 3.453124517261982; eval loss = 1.053575018963011
Epoch 1: train loss = 0.8335374938994646; eval loss = 0.6691032446699567
Epoch 2: train loss = 0.5963589655905962; eval loss = 0.5185086416209689
Epoch 3: train loss = 0.4807620374113321; eval loss = 0.4317912969627593
Epoch 4: train loss = 0.41476871917396785; eval loss = 0.3781255329894547
Epoch 5: train loss = 0.3737703591771424; eval loss = 0.3422349968500952
Epoch 6: train loss = 0.34589777915924785; eval loss = 0.3165890046275488
Epoch 7: train loss = 0.325512070376426; eval loss = 0.29728591420346556
Epoch 8: train loss = 0.3097923653945327; eval loss = 0.2821948009847414
Epoch 9: train loss = 0.29722966071590784; eval loss = 0.2700064215255846
Epoch 10: train loss = 0.2869205148629844; eval loss = 0.25990360533210016
Epoch 11: train loss = 0.27829304997995497; eval loss = 0.25135789717003554
Epoch 12: train loss = 0.2709647073447704; eval loss = 0.24401833539146303
Epoch 13: train loss = 0.26466250626556576;

In [ ]:
# Save model
torch.save(model.state_dict(), 'models/rmgnv2_epoch100.pth')

Reconstruct CFD output

In [ ]:
# reconstruct CFD
def print_prediction(model, data):
    pass

In [ ]:
# Model initializing
model = RecurrentMeshGraphNet(
    input_dim_node = args.input_dim_node,
    input_dim_edge = args.input_dim_edge,
    output_dim_node = args.output_dim_node,
    output_dim_edge = args.output_dim_edge,
    hidden_dim = args.hidden_dim,
    n_processors = args.n_processors
)
model = model.to(args.device)
model.load_state_dict(torch.load('models/rmgnv2_epoch100.pth'))

In [ ]:
# Plot prediction/ground truth
import matplotlib.pyplot as plt

def plot_comparison(model, data):
    n_edge = data.edge_attr.size(0)
    hidden = torch.zeros(n_edge, args.hidden_dim).to(args.device)
    _x = data.x
    edge_index = data.edge_index.to(args.device)
    _edge_attr = data.edge_attr

    total_loss = 0
    total_node_out = [data.pressure[:,0].unsqueeze(1)]
    total_edge_out = [data.velocity[:,0].unsqueeze(1)]
    for i in range(1, args.n_time):
        model.eval()
        with torch.no_grad():
            _pressure = data.pressure[:,i-1].unsqueeze(1)
            _velocity = data.velocity[:,i-1].unsqueeze(1)
            _flowrate_bc = data.flowrate_bc[:,i].unsqueeze(1)
            x = torch.cat([_x, _pressure], dim=1).to(args.device)
            edge_attr = torch.cat([_edge_attr, _velocity, _flowrate_bc], dim=1).to(args.device)
            node_out, edge_out, hidden = model(x, edge_index, edge_attr, hidden)
            hidden = hidden.detach()


            loss = criterion(node_out, data.pressure[:,i].unsqueeze(1).float().to(args.device))
            loss += criterion(edge_out, data.velocity[:,i].unsqueeze(1).float().to(args.device))

            total_loss += loss.item()
            total_node_out.append(node_out.detach().cpu())
            total_edge_out.append(edge_out.detach().cpu())
    total_node_out = torch.cat(total_node_out, dim=1)
    total_edge_out = torch.cat(total_edge_out, dim=1)
    
    # plot
    node = 50000
    y_pred = total_edge_out[node].numpy()
    y_true = data.velocity[node].numpy()
    x = [i * 4.8 /200 for i in range(201)]
    plt.plot(x, y_pred, c='red', label='RMGN')
    plt.plot(x, y_true, c='blue', linestyle='dashdot', label='ground_truth')
    # plt.ylim([-50,50])
    plt.legend(loc='upper right')
    plt.ylabel('Pressure', fontsize=20)
    plt.xlabel('Time', fontsize=20)
    plt.show()
    
    return total_loss
    
mean, std = mean_std_dataset(_dataset, set_id=list(range(_dataset.len())))
plot_comparison(model, normalize(_dataset[40], mean, std))